New hires definition: anyone who got hired after 2011

In [4]:
import pandas as pd, duckdb
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import os
import polars as pl
from pathlib import Path
import re

In [2]:
df = pl.read_parquet("/kellogg/proj/lgg3230/UnionSpill/Data/RAIS_aux/worker_estab_lagos.parquet").to_pandas()

In [3]:
df.head()

,identificad,year,year_str,l_firm_emp,lr_remdezr,lr_remmedr,l_firm_emp_2009,l_firm_emp_2009_5,lr_remdezr_2009,lr_remmedr_2009,...,lr_remmedr_w,r_remmedr_w,r_remmedr_h_w,r_remdezr_h_w,lr_remdezr_w,r_remdezr_w,dtnascimento_stata_w,modeind_w,modemun_w,_merge
0,00001961000110,2009,2009,2.484907,6.891086,6.694481,2.484907,NaN,6.891086,6.694481,...,6.708473,819.318298,4.282629,1.454113,6.708018,818.946106,1986-10-30,23494,351230,3
1,00001961000110,2010,2010,2.079442,6.829744,6.741766,2.484907,NaN,6.891086,6.694481,...,6.729254,836.523193,4.372560,1.536467,6.790373,889.245239,1981-11-20,23494,351230,3
2,00001961000110,2011,2011,2.484907,6.827140,6.758582,2.484907,NaN,6.891086,6.694481,...,6.776421,876.924072,4.583738,1.574861,6.828767,924.050659,NaT,23494,351230,3
3,00001961000110,2012,2012,2.772589,6.858742,6.857368,2.484907,NaN,6.891086,6.694481,...,7.128697,1247.250488,6.519458,1.874791,7.128697,1247.250488,NaT,23494,351230,3
4,00001961000110,2013,2013,2.944439,6.935752,6.921747,2.484907,NaN,6.891086,6.694481,...,6.923235,1015.599915,5.308605,1.669329,6.923235,1015.599915,NaT,23494,351230,3


In [11]:
pattern = re.compile(r"(remdezr|remmedr).*_w$", re.IGNORECASE)
cols_wage = [col for col in df.columns if pattern.search(col)]
print(cols_wage)

['remdezr_w', 'remmedr_w', 'lr_remmedr_w', 'r_remmedr_w', 'r_remmedr_h_w', 'r_remdezr_h_w', 'lr_remdezr_w', 'r_remdezr_w']


In [25]:
df[['dtadmissao_stata_w']].dtypes

dtadmissao_stata_w    datetime64[ns]
dtype: object

In [14]:
var = "remdezr_w"

n_unique = (
    df
    .groupby(["identificad", "year"])[var]
    .nunique(dropna=False)
    .reset_index(name="n_unique")
)

In [15]:
n_unique["has_variation"] = n_unique["n_unique"] > 1
n_unique["has_variation"].value_counts()

# lr_remdezr is at the firm level.

has_variation
True     136780
False      3993
Name: count, dtype: int64

In [16]:
len(df)

20303610

In [17]:
76/20303

0.0037432891690883123

In [18]:
cutoff_year = 2011
year_vals = pd.to_numeric(df['year'], errors='coerce')
tenure_months = pd.to_numeric(df['tempempr_w'], errors='coerce')
months_since_hire = ((tenure_months - 1).clip(lower=0)) // 12
hire_year_from_tenure = year_vals - months_since_hire

df['new_hire_after2011_temp'] = hire_year_from_tenure > cutoff_year
df['new_hire_after2011_dt'] = df['dtadmissao_stata_w'].dt.year > cutoff_year

agree_mask = df['new_hire_after2011_temp'].fillna(False) == df['new_hire_after2011_dt'].fillna(False)
print(f'Definitions agree for all rows: {agree_mask.all()}')
print(f'Rows with disagreement: {(~agree_mask).sum()}')

df['new_hire_after2011'] = df['new_hire_after2011_dt'].where(agree_mask, df['new_hire_after2011_temp'])
df[['new_hire_after2011_temp', 'new_hire_after2011_dt', 'new_hire_after2011']].head()


Definitions agree for all rows: False
Rows with disagreement: 76152


,new_hire_after2011_temp,new_hire_after2011_dt,new_hire_after2011
0,False,False,False
1,False,False,False
2,False,False,False
3,True,True,True
4,True,True,True


In [19]:
df['year'] = pd.to_numeric(df['year'], errors='coerce')
df['firm_emp'] = pd.to_numeric(df['firm_emp'], errors='coerce')
df['new_hire_after2011'] = df['new_hire_after2011_temp'].fillna(False).astype(bool)
print(f"Share of spells flagged as new hires (tenure definition): {df['new_hire_after2011'].mean():.4f}")


Share of spells flagged as new hires (tenure definition): 0.2660


In [20]:
output_dir = Path('/kellogg/proj/lgg3230/UnionSpill/Data/RAIS_aux')
dta_path_worker = output_dir / 'worker_year_pre_new_vs_nonnew.dta'
df_lagos_worker = df[['PIS_w', 'year', 'firm_emp', 'new_hire_after2011', 
                     'remdezr_w', 'remmedr_w', 'lr_remmedr_w', 'r_remmedr_w', 'r_remmedr_h_w', 'r_remdezr_h_w', 'lr_remdezr_w', 'r_remdezr_w', 'microregion', 'industry1',
                       'mode_base_month', 'totalflows_n', 'totaltreat_pw_n','identificad_w',
                     'treat_ultra', 'treat_year']]



In [21]:
df_lagos_worker.to_stata(dta_path_worker, write_index=False)



In [32]:
yearly_counts = (
    df.groupby('year')['new_hire_after2011']
      .agg(['sum', 'count'])
      .rename(columns={'sum': 'new_hires', 'count': 'total_spells'})
      .reset_index()
)
yearly_counts['pct_of_employment'] = 100 * yearly_counts['new_hires'] / yearly_counts['total_spells']
display(yearly_counts)


,year,new_hires,total_spells,pct_of_employment
0,2009,0,2405042,0.000000
1,2010,0,2576717,0.000000
2,2011,0,2688398,0.000000
3,2012,673977,2681525,25.134093
4,2013,1032269,2668941,38.677101
5,2014,1223826,2615406,46.792964
6,2015,1238911,2417538,51.246806
7,2016,1231040,2250043,54.711843


In [33]:
firm_year_share = (
    df.groupby(['identificad', 'year'])
      .agg(total_workers=('new_hire_after2011', 'size'),
           new_hires=('new_hire_after2011', 'sum'))
      .reset_index()
)
firm_year_share['pct_new_hires'] = 100 * firm_year_share['new_hires'] / firm_year_share['total_workers']
firm_year_avg = (
    firm_year_share.groupby('year')['pct_new_hires']
      .mean()
      .reset_index(name='avg_pct_new_hires_per_firm')
)
display(firm_year_avg)

if 'firm_emp_pre' not in df.columns:
    pre_emp = (
        df[df['year'].between(2009, 2011)]
          .groupby('identificad')['firm_emp']
          .mean()
    )
    df['firm_emp_pre'] = df['identificad'].map(pre_emp)

firm_pre = (
    df[['identificad', 'firm_emp_pre']]
      .drop_duplicates()
      .set_index('identificad')
)
pretreat_share = firm_year_share[firm_year_share['year'].between(2009, 2011)]
firm_share_pre = (
    pretreat_share.groupby('identificad')['pct_new_hires']
      .mean()
      .to_frame('avg_pct_new_hires_pre')
)
firm_share_pre = firm_share_pre.join(firm_pre)
corr_df = firm_share_pre.dropna()
corr_val = corr_df['avg_pct_new_hires_pre'].corr(corr_df['firm_emp_pre'])
print(f"Correlation between pretreatment new hire share and firm_emp_pre: {corr_val:.4f}")
display(firm_share_pre.head())


,year,avg_pct_new_hires_per_firm
0,2009,0.000000
1,2010,0.000000
2,2011,0.000000
3,2012,24.250475
4,2013,37.533844
5,2014,45.210006
6,2015,49.755993
7,2016,52.686702


Correlation between pretreatment new hire share and firm_emp_pre: nan


/home/lgg3230/.conda/envs/venv_python312/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/lgg3230/.conda/envs/venv_python312/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3066: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


,avg_pct_new_hires_pre,firm_emp_pre
identificad,,
00001961000110,0.0,11.000000
00009638000355,0.0,274.057072
00009638000436,0.0,49.200000
00012377000160,0.0,232.147193
00018215000130,0.0,58.976608


In [34]:
df['r_remdezr_w'] = pd.to_numeric(df['r_remdezr_w'], errors='coerce')
post = df[df['year'] >= 2012]
post_new = post[post['new_hire_after2011']]
post_non = post[~post['new_hire_after2011']]
firm_post_wages = (
    pd.concat([
        post_new.groupby('identificad')['r_remdezr_w'].mean().rename('avg_wage_new_hires'),
        post_non.groupby('identificad')['r_remdezr_w'].mean().rename('avg_wage_non_new')
    ], axis=1)
      .reset_index()
)
display(firm_post_wages.head())
print(f"Overall new hire avg wage (post-2012): {post_new['r_remdezr_w'].mean():.2f}")
print(f"Overall non-new hire avg wage (post-2012): {post_non['r_remdezr_w'].mean():.2f}")


,identificad,avg_wage_new_hires,avg_wage_non_new
0,00001961000110,1022.773132,1018.429382
1,00009638000355,3980.837402,3398.070312
2,00009638000436,2447.232910,3351.314453
3,00012377000160,2577.585449,3573.794434
4,00018215000130,1218.586548,2194.821045


Overall new hire avg wage (post-2012): 2438.24
Overall non-new hire avg wage (post-2012): 4264.54


In [35]:
from pathlib import Path
cutoff_year = 2011
df['placebo_year'] = (df['year'] <= cutoff_year).astype(int)
df['pre_or_newhire'] = ((df['year'] <= cutoff_year) | (df['new_hire_after2011'])).astype(int)
df['pre_or_nonnew'] = ((df['year'] <= cutoff_year) | (~df['new_hire_after2011'])).astype(int)

firm_level_cols = [
    'lagos_sample_avg', 'treat_ultra', 'treat_year', 'in_balanced_panel',
    'firm_emp', 'totaltreat_pw_n', 'totaltreat_pf_n', 'intreat_n',
    'outtreat_n', 'totalflows_n', 'lr_remdezr', 'industry1',
    'mode_base_month', 'microregion', 'placebo_year'
]
agg_map = {col: 'first' for col in firm_level_cols}

output_dir = Path('/kellogg/proj/lgg3230/UnionSpill/Data/RAIS_aux')

def collapse_group(mask_col: str, output_name: str):
    subset = df[df[mask_col] == 1]
    collapsed = (
        subset.groupby(['identificad', 'year'], as_index=False)
              .agg(agg_map)
    )
    collapsed['lr_remdezr_first'] = collapsed['lr_remdezr']
    out_path = output_dir / output_name
    collapsed.to_parquet(out_path, index=False)
    print(f"Saved {len(collapsed)} rows to {out_path}")
    return collapsed

pre_new_df = collapse_group('pre_or_newhire', 'firm_year_pre_or_newhire.parquet')
pre_non_df = collapse_group('pre_or_nonnew', 'firm_year_pre_or_nonnew.parquet')
display(pre_new_df.head())
display(pre_non_df.head())


Saved 133166 rows to /kellogg/proj/lgg3230/UnionSpill/Data/RAIS_aux/firm_year_pre_or_newhire.parquet
Saved 138076 rows to /kellogg/proj/lgg3230/UnionSpill/Data/RAIS_aux/firm_year_pre_or_nonnew.parquet


,identificad,year,lagos_sample_avg,treat_ultra,treat_year,in_balanced_panel,firm_emp,totaltreat_pw_n,totaltreat_pf_n,intreat_n,outtreat_n,totalflows_n,lr_remdezr,industry1,mode_base_month,microregion,placebo_year,lr_remdezr_first
0,00001961000110,2009,1,1,0,1,12,0.05,1.0,0.0,2.0,2.0,6.891086,1234,5,35023,1,6.891086
1,00001961000110,2010,1,1,0,1,8,0.05,1.0,0.0,2.0,2.0,6.829744,1234,5,35023,1,6.829744
2,00001961000110,2011,1,1,0,1,12,0.05,1.0,0.0,2.0,2.0,6.827140,1234,5,35023,1,6.827140
3,00001961000110,2012,1,1,1,1,16,0.05,1.0,0.0,2.0,2.0,6.858742,1234,5,35023,0,6.858742
4,00001961000110,2013,1,1,1,1,19,0.05,1.0,0.0,2.0,2.0,6.935752,1234,5,35023,0,6.935752


,identificad,year,lagos_sample_avg,treat_ultra,treat_year,in_balanced_panel,firm_emp,totaltreat_pw_n,totaltreat_pf_n,intreat_n,outtreat_n,totalflows_n,lr_remdezr,industry1,mode_base_month,microregion,placebo_year,lr_remdezr_first
0,00001961000110,2009,1,1,0,1,12,0.05,1.0,0.0,2.0,2.0,6.891086,1234,5,35023,1,6.891086
1,00001961000110,2010,1,1,0,1,8,0.05,1.0,0.0,2.0,2.0,6.829744,1234,5,35023,1,6.829744
2,00001961000110,2011,1,1,0,1,12,0.05,1.0,0.0,2.0,2.0,6.827140,1234,5,35023,1,6.827140
3,00001961000110,2012,1,1,1,1,16,0.05,1.0,0.0,2.0,2.0,6.858742,1234,5,35023,0,6.858742
4,00001961000110,2013,1,1,1,1,19,0.05,1.0,0.0,2.0,2.0,6.935752,1234,5,35023,0,6.935752


In [36]:
from pathlib import Path
cutoff_year = 2011
req_cols = ['pre_or_newhire', 'pre_or_nonnew', 'placebo_year']
if any(col not in df.columns for col in req_cols):
    df['placebo_year'] = (df['year'] <= cutoff_year).astype(int)
    df['pre_or_newhire'] = ((df['year'] <= cutoff_year) | (df['new_hire_after2011'])).astype(int)
    df['pre_or_nonnew'] = ((df['year'] <= cutoff_year) | (~df['new_hire_after2011'])).astype(int)
group_cols = ['identificad', 'year']
keep_cols = [
    'lagos_sample_avg', 'treat_ultra', 'treat_year', 'in_balanced_panel',
    'firm_emp', 'totaltreat_pw_n', 'totaltreat_pf_n', 'intreat_n',
    'outtreat_n', 'totalflows_n', 'lr_remdezr', 'industry1',
    'mode_base_month', 'microregion'
]
agg_map = {col: 'first' for col in keep_cols}
firm_year = df.groupby(group_cols, as_index=False).agg(agg_map)
avg_pre_new = (
    df[df['pre_or_newhire'] == 1]
      .groupby(group_cols)['lr_remdezr']
      .mean()
      .rename('avg_lr_remdezr_pre_or_newhire')
)
avg_pre_non = (
    df[df['pre_or_nonnew'] == 1]
      .groupby(group_cols)['lr_remdezr']
      .mean()
      .rename('avg_lr_remdezr_pre_or_nonnew')
)
firm_year = firm_year.merge(avg_pre_new, on=group_cols, how='left')
firm_year = firm_year.merge(avg_pre_non, on=group_cols, how='left')
bool_cols = firm_year.select_dtypes(include=['bool']).columns
if len(bool_cols) > 0:
    firm_year[bool_cols] = firm_year[bool_cols].astype(int)
output_dir = Path('/kellogg/proj/lgg3230/UnionSpill/Data/RAIS_aux')
parquet_path = output_dir / 'firm_year_pre_new_vs_nonnew.parquet'
dta_path = output_dir / 'firm_year_pre_new_vs_nonnew.dta'
firm_year.to_parquet(parquet_path, index=False)
firm_year.to_stata(dta_path, write_index=False)
row_count = len(firm_year)
print(f'Saved {row_count} firm-year rows to {parquet_path}')
print(f'Also saved Stata version to {dta_path}')
print(f'Row count: {row_count}')
display(firm_year.head())


Saved 140773 firm-year rows to /kellogg/proj/lgg3230/UnionSpill/Data/RAIS_aux/firm_year_pre_new_vs_nonnew.parquet
Also saved Stata version to /kellogg/proj/lgg3230/UnionSpill/Data/RAIS_aux/firm_year_pre_new_vs_nonnew.dta
Row count: 140773


,identificad,year,lagos_sample_avg,treat_ultra,treat_year,in_balanced_panel,firm_emp,totaltreat_pw_n,totaltreat_pf_n,intreat_n,outtreat_n,totalflows_n,lr_remdezr,industry1,mode_base_month,microregion,avg_lr_remdezr_pre_or_newhire,avg_lr_remdezr_pre_or_nonnew
0,00001961000110,2009,1,1,0,1,12,0.05,1.0,0.0,2.0,2.0,6.891086,1234,5,35023,6.891085,6.891085
1,00001961000110,2010,1,1,0,1,8,0.05,1.0,0.0,2.0,2.0,6.829744,1234,5,35023,6.829744,6.829744
2,00001961000110,2011,1,1,0,1,12,0.05,1.0,0.0,2.0,2.0,6.827140,1234,5,35023,6.827140,6.827140
3,00001961000110,2012,1,1,1,1,16,0.05,1.0,0.0,2.0,2.0,6.858742,1234,5,35023,6.858742,6.858742
4,00001961000110,2013,1,1,1,1,19,0.05,1.0,0.0,2.0,2.0,6.935752,1234,5,35023,6.935752,6.935752


In [37]:
# Build firm-year panel with incumbent vs new hire wages for interaction regressions
group_cols = ['identificad', 'year']
keep_cols = [
    'lagos_sample_avg', 'treat_ultra', 'treat_year', 'in_balanced_panel',
    'firm_emp', 'totaltreat_pw_n', 'totaltreat_pf_n', 'intreat_n',
    'outtreat_n', 'totalflows_n', 'lr_remdezr', 'industry1',
    'mode_base_month', 'microregion'
]
agg_map = {col: 'first' for col in keep_cols}

firm_level = df.groupby(group_cols, as_index=False).agg(agg_map)

wage_pivot = (
    df.groupby(group_cols + ['new_hire_after2011'])['lr_remdezr']
      .mean()
      .reset_index()
      .pivot_table(index=group_cols, columns='new_hire_after2011', values='lr_remdezr')
      .rename(columns={False: 'avg_lr_remdezr_incumbent', True: 'avg_lr_remdezr_new_hire'})
      .reset_index()
)
for col in ['avg_lr_remdezr_incumbent', 'avg_lr_remdezr_new_hire']:
    if col not in wage_pivot.columns:
        wage_pivot[col] = pd.NA

firm_year_long = wage_pivot.melt(
    id_vars=group_cols,
    value_vars=['avg_lr_remdezr_incumbent', 'avg_lr_remdezr_new_hire'],
    var_name='wage_group',
    value_name='avg_lr_remdezr'
)
firm_year_long['wage_type'] = firm_year_long['wage_group'].map({
    'avg_lr_remdezr_incumbent': 'incumbent',
    'avg_lr_remdezr_new_hire': 'new_hire'
})
firm_year_long['D_incumbent'] = (firm_year_long['wage_type'] == 'incumbent').astype(int)
firm_year_long['new_hire_after2011'] = (firm_year_long['wage_type'] == 'new_hire').astype(int)
firm_year_long = firm_year_long.merge(firm_level, on=group_cols, how='left')
bool_cols = firm_year_long.select_dtypes(include=['bool']).columns
if len(bool_cols) > 0:
    firm_year_long[bool_cols] = firm_year_long[bool_cols].astype(int)

output_dir = Path('/kellogg/proj/lgg3230/UnionSpill/Data/RAIS_aux')
parquet_path = output_dir / 'firm_year_incumbent_vs_newhire_long.parquet'
dta_path = output_dir / 'firm_year_incumbent_vs_newhire_long.dta'
firm_year_long.to_parquet(parquet_path, index=False)
firm_year_long.to_stata(dta_path, write_index=False)
print(f'Saved {len(firm_year_long)} firm-year group rows to {parquet_path}')
print(f'Also saved Stata version to {dta_path}')
display(firm_year_long.head())


Saved 279902 firm-year group rows to /kellogg/proj/lgg3230/UnionSpill/Data/RAIS_aux/firm_year_incumbent_vs_newhire_long.parquet
Also saved Stata version to /kellogg/proj/lgg3230/UnionSpill/Data/RAIS_aux/firm_year_incumbent_vs_newhire_long.dta


,identificad,year,wage_group,avg_lr_remdezr,wage_type,D_incumbent,new_hire_after2011,lagos_sample_avg,treat_ultra,treat_year,...,firm_emp,totaltreat_pw_n,totaltreat_pf_n,intreat_n,outtreat_n,totalflows_n,lr_remdezr,industry1,mode_base_month,microregion
0,00001961000110,2009,avg_lr_remdezr_incumbent,6.891085,incumbent,1,0,1,1,0,...,12,0.05,1.0,0.0,2.0,2.0,6.891086,1234,5,35023
1,00001961000110,2010,avg_lr_remdezr_incumbent,6.829744,incumbent,1,0,1,1,0,...,8,0.05,1.0,0.0,2.0,2.0,6.829744,1234,5,35023
2,00001961000110,2011,avg_lr_remdezr_incumbent,6.827140,incumbent,1,0,1,1,0,...,12,0.05,1.0,0.0,2.0,2.0,6.827140,1234,5,35023
3,00001961000110,2012,avg_lr_remdezr_incumbent,6.858742,incumbent,1,0,1,1,1,...,16,0.05,1.0,0.0,2.0,2.0,6.858742,1234,5,35023
4,00001961000110,2013,avg_lr_remdezr_incumbent,6.935752,incumbent,1,0,1,1,1,...,19,0.05,1.0,0.0,2.0,2.0,6.935752,1234,5,35023


In [38]:
# Average lr_remdezr by year
avg_lr_by_year = (
    firm_year_long.groupby('year')['avg_lr_remdezr']
      .mean()
      .reset_index(name='avg_lr_remdezr')
)
display(avg_lr_by_year)


,year,avg_lr_remdezr
0,2009,7.699495
1,2010,7.734018
2,2011,7.763325
3,2012,7.789550
4,2013,7.832594
5,2014,7.857624
6,2015,7.851878
7,2016,7.874547
